In [104]:
import numpy as np
import pandas as pd

## Features created in this file:
### number of businesses, number of employed persons, first quarter pay, and annual pay per zipcode

## to get dataframe that I created in this file:
#### df = pd.read_pickle('business_patterns.pkl')

In [73]:
bus_data_raw = pd.read_csv('BP_2010_00CZ1_with_ann.csv')

In [74]:
bus_data_raw.head(4)

,GEO.id,GEO.id2,GEO.display-label,NAICS.id,NAICS.display-label,YEAR.id,ESTAB,EMP,PAYQTR1,PAYANN
0,Geographic identifier code,Id2,Geographic area name,2007 NAICS code,Meaning of 2007 NAICS code,Year,Number of establishments,Paid employees for pay period including March ...,"First-quarter payroll ($1,000)","Annual payroll ($1,000)"
1,8610000US00501,00501,"ZIP 00501 (Holtsville, NY)",00,Total for all sectors,2010,2,a,D,D
2,8610000US01001,01001,"ZIP 01001 (Agawam, MA)",00,Total for all sectors,2010,448,9185,70693,314508
3,8610000US01002,01002,"ZIP 01002 (Amherst, MA)",00,Total for all sectors,2010,570,15633,82069,351589


In [88]:
# rename columns and drop unnecessary columns
bus_data = bus_data_raw.drop(['GEO.id2','NAICS.id','NAICS.display-label', 'YEAR.id'],1)
bus_data.columns = bus_data.iloc[0].values
bus_data = bus_data[bus_data['Geographic identifier code'] != 'Geographic identifier code']
bus_data = bus_data.reset_index(drop=True)

# code from census data that maps letters (in paid employees col) to ints
letter_map = {'a':10, 'D':np.nan, 'b':60, 'c':175, 'h':3750, 'f':750, 'i':7500, 'g':1750, 'S':np.nan, 'e':375, 'j':17500, 'l':75000, 'k':37500, 'm':100000}
# get list of paid employees column for processing
paid_employees_raw = bus_data['Paid employees for pay period including March 12 (number)']

# function to process the paid employees. turn strings to ints, and convert letters to corresponding ints.
def process_employees(el):
    try:
        num = int(el)
    except ValueError:
        num = letter_map[el]
    return num

# get list of paid employees, all values should be ints
paid_employees = [process_employees(el) for el in paid_employees_raw]

# do the same for 'first-quarter payroll' and 'annual payroll', they only have the 'D' value
first_quarter = [process_employees(el) for el in bus_data['First-quarter payroll ($1,000)']]
annual = [process_employees(el) for el in bus_data['Annual payroll ($1,000)']]

# add these to dataframe, delete old columns
bus_data['paid_employees'] = paid_employees
bus_data['first_quarter_payroll_1000'] = first_quarter
bus_data['annual_payroll_1000'] = annual
# get zipcodes from last 5 characters of 'Geographic identifier code'
bus_data['zipcode'] = [el[-5:] for el in bus_data['Geographic identifier code']]
bus_data = bus_data.drop(['Paid employees for pay period including March 12 (number)','First-quarter payroll ($1,000)','Annual payroll ($1,000)','Geographic identifier code'],1)

In [91]:
bus_data = bus_data.dropna(subset = ['annual_payroll_1000', 'first_quarter_payroll_1000'])

In [92]:
bus_data.shape, bus_data_raw.shape

((31255, 6), (39105, 10))

In [93]:
bus_data.to_pickle('business_patterns.pkl')

In [77]:
bus_data.head(4)

,Geographic area name,Number of establishments,paid_employees,first_quarter_payroll_1000,annual_payroll_1000,zipcode
0,"ZIP 00501 (Holtsville, NY)",2,10,NaN,NaN,00501
1,"ZIP 01001 (Agawam, MA)",448,9185,70693,314508,01001
2,"ZIP 01002 (Amherst, MA)",570,15633,82069,351589,01002
3,"ZIP 01003 (Amherst, MA)",16,168,1297,5761,01003
